# Projeto final - SME0130 Redes Complexas

**Professor Francisco Rodrigues**

- **Arthur Vergaças Daher Martins | 12542672**
- **Gustavo Sampaio Lima | 12623992**
- **João Pedro Duarte Nunes | 12542460**
- **Pedro Guilherme dos Reis Teixeira | 12542477**

## Tópico escolhido:

<p style="text-align: center; font-size: 24px">3 – Como a cooperação é influenciada pela topologia da rede?</p>


In [1]:
import numpy as np
import networkx as nx
import random
from enum import Enum, auto
from typing import Self, Callable

In [2]:
class Strategy(Enum):
    COOPERATOR = auto()
    DEFECTOR = auto()

In [4]:
class UpdateRule(Enum):
    REP = auto()
    UI = auto()
    MOR = auto()

    def __str__(self) -> str:
        match self:
            case self.REP:
                return "Replicator Dynamics"
            case self.UI:
                return "Unconditional Imitation"
            case self.MOR:
                return "Moran rule"

    def update(
        self,
        player,
        player_node: int,
        players: list,
        network: nx.Graph,
        temptation_to_defect_payoff: float,
        handle_chosen_target: Callable,
    ):
        match self:
            case UpdateRule.REP:
                chosen_neighbor_index = random.choice(network.neighbors(player_node))
                chosen_neighbor = players[chosen_neighbor_index]

                player_degree = network.degree[player_node]  # type: ignore
                chosen_neighbor_degree = network.degree[chosen_neighbor_index]  # type: ignore

                probability_to_change = (
                    (chosen_neighbor.current_payoff - player.current_payoff)
                    / temptation_to_defect_payoff
                    * max(player_degree, chosen_neighbor_degree)
                )

                if random.random() > probability_to_change:
                    handle_chosen_target(chosen_neighbor)
                else:
                    handle_chosen_target(player)

            case UpdateRule.UI:
                neighbors = [players[n] for n in network.neighbors(player_node)]

                best_neighbor = max(neighbors, key=lambda n: n.current_payoff)

                if best_neighbor.current_payoff > player.current_payoff:
                    handle_chosen_target(best_neighbor)
                else:
                    handle_chosen_target(player)

            case UpdateRule.MOR:
                neighbors = [players[n] for n in network.neighbors(player_node)]

                neighbors_and_player = [*neighbors, player]

                sum_of_payoffs = sum([p.current_payoff for p in neighbors_and_player])

                weights = [
                    p.current_payoff / sum_of_payoffs for p in neighbors_and_player
                ]

                chosen_target = random.choices(neighbors_and_player, weights=weights)[0]

                handle_chosen_target(chosen_target)

In [ ]:
class NetworkType(Enum):
    ER = auto()
    WS_0 = auto()
    WS_005 = auto()
    WS_01 = auto()
    WS_03 = auto()
    BA = auto()
    NL_BA = auto()

    def __str__(self) -> str:
        match self:
            case self.ER:
                return "Erdős-Rényi Network Type"
            case self.WS_0:
                return "Watts-Strogatz (Small-World) Network Type (p = 0)"
            case self.WS_005:
                return "Watts-Strogatz (Small-World) Network Type (p = 0.05)"
            case self.WS_01:
                return "Watts-Strogatz (Small-World) Network Type (p = 0.1)"
            case self.WS_03:
                return "Watts-Strogatz (Small-World) Network Type (p = 0.3)"
            case self.BA:
                return "Barabási-Albert Network Type"
            case self.NL_BA:
                return "Non-linear Barabási-Albert Network Type"

    def generator_function(
        self, number_of_nodes: int, average_degree: float
    ) -> Callable[[], nx.Graph]:
        match self:
            case self.ER:
                return lambda: nx.fast_gnp_random_graph(
                    number_of_nodes, average_degree / (number_of_nodes - 1)
                )
            case self.WS_0:
                return lambda: nx.watts_strogatz_graph(
                    number_of_nodes, int(average_degree), 0
                )
            case self.WS_005:
                return lambda: nx.watts_strogatz_graph(
                    number_of_nodes, int(average_degree), 0.05
                )
            case self.WS_01:
                return lambda: nx.watts_strogatz_graph(
                    number_of_nodes, int(average_degree), 0.1
                )
            case self.WS_03:
                return lambda: nx.watts_strogatz_graph(
                    number_of_nodes, int(average_degree), 0.3
                )
            case self.BA:
                return lambda: nx.barabasi_albert_graph(
                    number_of_nodes, int(average_degree / 2)
                )
            case self.NL_BA:
                return (
                    lambda: nx.Graph()
                )  # TODO não achei um gerador pra esse cara aqui

In [ ]:
class Player:
    def __init__(self, strategy: Strategy, update_rule: UpdateRule):
        self.strategy = strategy
        self.update_rule = update_rule
        self.current_payoff = 0

    @classmethod
    def from_fractions(
        cls,
        cooperators_fraction: float,
        rule_a_fraction: float,
        update_rules: list[UpdateRule],
    ) -> Self:
        strategy = random.choices(
            [Strategy.COOPERATOR, Strategy.DEFECTOR],
            weights=[cooperators_fraction, 1 - cooperators_fraction],
        )[0]

        update_rule = random.choices(
            update_rules, weights=[rule_a_fraction, 1 - rule_a_fraction]
        )[0]

        return cls(strategy, update_rule)

    def play_with(
        self,
        other: Self,
        temptation_to_defect_payoff: float,
        mutual_cooperation_payoff=1.0,
    ):
        # Seguindo o dilema do prisioneiro fraco (Nowak and May), que pode ser descrito da seguinte forma:
        #
        #    | C | D |
        #  C | x | 0 |
        #  D | b | 0 |
        #
        # Onde as linhas indicam a estratégia do jogador, e as colunas a estratégia do seu adversário.
        # Usualmente, x = 1.
        #

        if (
            self.strategy == Strategy.COOPERATOR
            and other.strategy == Strategy.COOPERATOR
        ):
            self.current_payoff += mutual_cooperation_payoff
        elif (
            self.strategy == Strategy.DEFECTOR and other.strategy == Strategy.COOPERATOR
        ):
            self.current_payoff += temptation_to_defect_payoff

    def evolve(
        self,
        self_node: int,
        players: list[Self],
        network: nx.Graph,
        temptation_to_defect_payoff: float,
    ):
        self.update_rule.update(
            self,
            self_node,
            players,
            network,
            temptation_to_defect_payoff,
            lambda target: self._copy(target),
        )

    def _copy(self, other: Self):
        self.strategy = other.strategy
        self.update_rule = other.update_rule

In [ ]:
def simulate_prisoners_dilemma(
    number_of_iterations=4000,
    number_of_players=5000,
    temptation_payoff=1,
    cooperators_fraction=0.5,
    rule_a_fraction=0.5,
    update_rules=[UpdateRule.MOR, UpdateRule.UI],
    network_type: NetworkType = NetworkType.BA,
    average_network_degree=6,
):
    cooperators_fraction_at_iteration = np.array([])
    # TODO talvez guardar também a fração de regras

    network = network_type.generator_function(
        number_of_players, average_network_degree
    )()

    players: list[Player] = [
        Player.from_fractions(cooperators_fraction, rule_a_fraction, update_rules)
        for _ in range(number_of_players)
    ]

    for _ in range(number_of_iterations):
        # jogar contra os vizinhos
        for node in network:
            player = players[node]
            neighbors = [players[n] for n in network.neighbors(node)]

            for adversary in neighbors:
                player.play_with(adversary, temptation_payoff)

        # evoluir de acordo com vizinhos
        for node in network:
            player = players[node]
            neighbors = [players[n] for n in network.neighbors(node)]

            player.evolve(neighbors)

        # armazenar informações sobre o estado da rede
        number_of_cooperators = 0
        for node in network:
            player = players[node]

            number_of_cooperators += player.strategy == Strategy.COOPERATOR

        np.append(
            cooperators_fraction_at_iteration, number_of_cooperators / number_of_players
        )